# Import Dependensi

Package

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Modul Perhitungan ET

In [22]:
from modul.gravimetri import gravimetri
from modul.bc_model import bc_model
from modul.hs_model import hs_model
from modul.pm_model import pm_model

Variabel

In [23]:
from variabel import variabel as var

# Import Dataset

In [24]:
# Import file excel
dataset_primer = 'data/dataset_primer_1.xlsx'
dataset_sekunder = 'data/dataset_sekunder.xlsx'

# Membaca semua sheet dari file Excel
data_primer = pd.ExcelFile(dataset_primer)
data_sekunder = pd.ExcelFile(dataset_sekunder)

# Fungsi-Fungsi

In [25]:
# Menghitung luas area di dalam variabel
def hitung_luas_media(sisi):
        cm_to_mm = sisi[0] * 10
        luas = (cm_to_mm ** 2)
        return luas              # Hasil dalam satuan mm

In [26]:
# Menghitung tanggal pengambilan data
def hitung_tanggal_data(day):
    day = int(day.split()[1])

    tahun = '2024'

    if day > 10:
        bulan = '11'
    else:
        bulan = '10'
    
    def hitung_tanggal(input_day):
        # Jika x antara 1 dan 31, y adalah x + 20, tetapi melingkar kembali ke awal setelah 31
        return (input_day + 21) % 31 if (input_day + 21) % 31 != 0 else 31

    hari = hitung_tanggal(day)
    hari = f"{hari:02d}"

    tanggal = (f"{tahun}-{bulan}-{hari}")

    return tanggal

# ET Model Gravimetri/Penimbangan

Perhitungan Gravimetri

In [27]:
hasil_gravimetri = []

for sheet_name in data_primer.sheet_names:
    # Baca data primer dan sekunder dari sheet excel
    df_data_primer = data_primer.parse(sheet_name)
    df_data_sekunder = data_sekunder.parse(sheet_name)
    
    # Logika program Anda
    for kode in var.sisi_media_tanaman.keys():
        kode_berat = f'berat_{kode}'
        kode_irigasi = f'supply_air_{kode}'
        
        # Buat array untuk variabel yang akan digunakan
        waktu = df_data_primer['waktu'].array
        berat = df_data_primer[kode_berat].array                            # Satuan kg
        irigasi_liter = df_data_primer[kode_irigasi].fillna(0).array        # Satuan liter
        curah_hujan = df_data_sekunder['curah_hujan'].array                 # Satuan mm

        # Hitung luas alas media tanaman
        luas_media = hitung_luas_media(var.sisi_media_tanaman[kode])    # Satuan mm^2

        # Ubah irigasi ke satuan mm
        irigasi_mm = (irigasi_liter * 1e6) / (luas_media)               # Satuan mm

        # Ubah berat ke satuan volume mm^3
        volume = berat * 1e6                                            # Satuan mm^3

        # Hitung total supply air
        supply_air = irigasi_mm + curah_hujan                           # Satuan mm

        perubahan_berat, perubahan_et, et_total = evapotranspirasi = gravimetri(berat=volume,
                                                                                waktu=waktu,
                                                                                luas=luas_media,
                                                                                supply_air=supply_air
                                                                                )

        # Simpan hasil ke list
        hasil_gravimetri.append({
            "Day": sheet_name,
            "Kode": kode,
            "ET Total": et_total
        })

# Buat DataFrame dari hasil
df_hasil_gravimetri = pd.DataFrame(hasil_gravimetri)

# Pivot tabel dengan "Day" sebagai index dan "Kode" sebagai kolom
df_hasil_gravimetri = df_hasil_gravimetri.pivot(index="Day", columns="Kode", values="ET Total")


In [28]:
df_hasil_gravimetri.to_csv('hasil/ET Aktual Gravimetri.csv', index=True)
df_hasil_gravimetri.head()

Kode,A1,A2,A3,A4,B1,B2,B3,B4,C1,C2,...,D3,D4,E1,E2,E3,E4,F1,F2,F3,F4
Day,,,,,,,,,,,,,,,,,,,,,
day 1,2.56,3.072,2.656,6.592,0.0,0.0,0.0,0.0,3.248980,7.395918,...,7.5250,7.95,7.575309,8.148148,13.066667,7.664198,23.984,13.456,15.848,9.880
day 2,45.92,21.120,6.464,44.000,0.0,0.0,0.0,0.0,18.269388,11.608163,...,6.0125,1.80,4.414815,5.392593,9.580247,12.898765,32.240,4.824,0.456,73.768
day 3,0.00,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0000,0.00,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000,0.000
day 4,0.00,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0000,0.00,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000,0.000


# ET Model Blaney-Criddle

In [30]:
hasil_bc_model = []

for sheet_name in data_sekunder.sheet_names:
    # Baca data primer dan sekunder dari sheet excel
    df_data_sekunder = data_sekunder.parse(sheet_name)

    tanggal_data = hitung_tanggal_data(day = sheet_name)
    # print(f"{sheet_name} : {tanggal_data}")
    
    # Memasukkan nilai dataset ke dalam variabel
    suhu_udara = np.array(df_data_sekunder['suhu_udara'])
    # print(f"Suhu udara {sheet_name}: {suhu_udara}")

    # Menghitung rata-rata suhu udara
    mean_suhu_udara = np.mean(suhu_udara)
    # print(f"Rata-rata suhu udara: {mean_suhu_udara}")

    # Menghitung ET dengan model Blaney-Criddle
    et_model_bc = bc_model(var.persentase_hari_terang, mean_suhu_udara)
    # print(f"ET Blaney-Criddle: {et_model_bc}\n")

    # Simpan hasil ke list
    hasil_bc_model.append({
        "Day": sheet_name,
        "Tanggal": tanggal_data,
        "Mean Suhu Udara": mean_suhu_udara,
        "ET Total": et_model_bc
    })

# Buat DataFrame dari hasil
df_hasil_bc = pd.DataFrame(hasil_bc_model)

In [31]:
df_hasil_bc.to_csv('hasil/Estimasi ET Blaney-Criddle.csv', index=False)
df_hasil_bc.head()

,Day,Tanggal,Mean Suhu Udara,ET Total
0,day 1,2024-10-22,30.01500,5.887863
1,day 2,2024-10-23,30.94875,6.003835
2,day 3,2024-10-24,31.14000,6.027588
3,day 4,2024-10-25,30.80625,5.986136
4,day 5,2024-10-26,30.90750,5.998712


# ET Model Hargreaves-Samani

In [32]:
hasil_hs_model = []

for sheet_name in data_sekunder.sheet_names:
    # Baca data primer dan sekunder dari sheet excel
    df_data_sekunder = data_sekunder.parse(sheet_name)
    
    tanggal_data = hitung_tanggal_data(day = sheet_name)
    # print(f"{sheet_name} : {tanggal_data}")

    # Memasukkan nilai dataset ke dalam variabel
    suhu_udara = np.array(df_data_sekunder['suhu_udara'])

    # Menghitung rata-rata, max, min, dan rentang suhu udara
    max_suhu_udara = np.max(suhu_udara)
    # print(f"Maksimal suhu udara: {max_suhu_udara}")
    min_suhu_udara = np.min(suhu_udara)
    # print(f"Minimal suhu udara: {min_suhu_udara}")
    mean_suhu_udara = (max_suhu_udara + min_suhu_udara) / 2
    # print(f"Rata-rata suhu udara: {mean_suhu_udara}")

    # Menghitung ET dengan model Hargreaves-Samani
    radiasi_ekstrateresterial, et_model_hs, et_model_hs_modification = hs_model(t_max = max_suhu_udara, 
                                                                                t_min = min_suhu_udara, 
                                                                                t_avg = mean_suhu_udara, 
                                                                                latitude = var.latitude, 
                                                                                input_date = tanggal_data)
    # print(f"ET Hargreaves-Samani: {et_model_hs}")
    # print(f"Radiasi atmosfer: {radiasi_ekstrateresterial}")

    # Simpan hasil ke list
    hasil_hs_model.append({
        "Day": sheet_name,
        "Tanggal": tanggal_data,
        "Max Suhu Udara": max_suhu_udara,
        "Min Suhu Udara": min_suhu_udara,
        "Mean Suhu Udara": mean_suhu_udara,
        "Radiasi Extrateresterial": radiasi_ekstrateresterial,
        "ET Total": et_model_hs,
    })

# Buat DataFrame dari hasil
df_hasil_hs = pd.DataFrame(hasil_hs_model)

In [33]:
df_hasil_hs.to_csv('hasil/Estimasi ET Hargreaves-Samani.csv', index=False)
df_hasil_hs.head()

,Day,Tanggal,Max Suhu Udara,Min Suhu Udara,Mean Suhu Udara,Radiasi Extrateresterial,ET Total
0,day 1,2024-10-22,32.36,24.94,28.650,15.761121,4.586724
1,day 2,2024-10-23,33.47,25.65,29.560,15.769790,4.803622
2,day 3,2024-10-24,33.64,25.69,29.665,15.777863,4.856608
3,day 4,2024-10-25,33.05,25.68,29.365,15.785356,4.648746
4,day 5,2024-10-26,33.07,25.59,29.330,15.792283,4.681888


# ET Model Penman-Monteith

In [34]:
hasil_pm_model = []

for sheet_name in data_sekunder.sheet_names:
    # Baca data primer dan sekunder dari sheet excel
    df_data_sekunder = data_sekunder.parse(sheet_name)
    
    tanggal_data = hitung_tanggal_data(day = sheet_name)
    # print(f"{sheet_name} : {tanggal_data}")

    # Memasukkan nilai dataset ke dalam variabel
    suhu_udara = np.array(df_data_sekunder['suhu_udara'])
    kecepatan_angin = np.array(df_data_sekunder['kecepatan_angin'])
    kelembapan_relatif = np.array(df_data_sekunder['kelembapan_relatif'])
    
    # Menghitung rata-rata kecepatan angin dan kelembapan relatif
    mean_kecepatan_angin = np.mean(kecepatan_angin)
    # print(f"Rata-rata kecepatan angin: {mean_kecepatan_angin}")
    mean_kelembapan_relatif = np.mean(kelembapan_relatif)
    # print(f"Rata-rata kelembapan relatif: {mean_kelembapan_relatif}")

    # Menghitung rata-rata, max, min, dan rentang suhu udara
    max_suhu_udara = np.max(suhu_udara)
    # print(f"Maksimal suhu udara: {max_suhu_udara}")
    min_suhu_udara = np.min(suhu_udara)
    # print(f"Minimal suhu udara: {min_suhu_udara}")
    mean_suhu_udara = (max_suhu_udara + min_suhu_udara) / 2
    # print(f"Rata-rata suhu udara: {mean_suhu_udara}")
    
    # Menghitung ET dengan model Penman-monteith
    radiasi_ekstrateresterial, et_model_pm, total_et_model_pm = pm_model(RH = mean_kelembapan_relatif, 
                                                                         u_2 = kecepatan_angin, 
                                                                         T_hr = suhu_udara, 
                                                                         T_avg = mean_suhu_udara, 
                                                                         T_max = max_suhu_udara, 
                                                                         T_min = min_suhu_udara, 
                                                                         altitude = var.altitude, 
                                                                         latitude = var.latitude, 
                                                                         input_date = tanggal_data)
    # print(f"ET Penman-monteith {sheet_name}: {total_et_model_pm}")
    # print(f"Radiasi atmosfer: {radiasi_ekstrateresterial}")

    # Simpan hasil ke list
    hasil_pm_model.append({
        "Day": sheet_name,
        "Tanggal": tanggal_data,
        "Max Suhu Udara": max_suhu_udara,
        "Min Suhu Udara": min_suhu_udara,
        "Mean Suhu Udara": mean_suhu_udara,
        "Mean Kecepatan Angin": mean_kecepatan_angin,
        "Mean Kelembapan Relatif": mean_kelembapan_relatif,
        "Radiasi Extrateresterial": radiasi_ekstrateresterial,
        "ET Total": total_et_model_pm
    })

# Buat DataFrame dari hasil
df_hasil_pm = pd.DataFrame(hasil_pm_model)

In [35]:
df_hasil_pm.to_csv('hasil/Estimasi ET Penman-Monteith.csv', index=False)
df_hasil_pm.head()

,Day,Tanggal,Max Suhu Udara,Min Suhu Udara,Mean Suhu Udara,Mean Kecepatan Angin,Mean Kelembapan Relatif,Radiasi Extrateresterial,ET Total
0,day 1,2024-10-22,32.36,24.94,28.650,3.06625,50.91375,15.761121,3.450344
1,day 2,2024-10-23,33.47,25.65,29.560,3.07750,47.54000,15.769790,3.774212
2,day 3,2024-10-24,33.64,25.69,29.665,2.41000,47.71875,15.777863,3.315916
3,day 4,2024-10-25,33.05,25.68,29.365,2.83750,51.68875,15.785356,3.403469
4,day 5,2024-10-26,33.07,25.59,29.330,3.80000,48.86750,15.792283,4.104162
